In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_identity = pd.read_csv('/home/other/kaggle/data/train_identity.csv')
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [3]:
# for col in train_identity.columns:
#     if type(train_identity[col].iloc[0]) == np.float64:
#         train_identity[col] = train_identity[col].astype(np.float16)
# #train_identity.to_csv('/kaggle/low_dim/train_transaction.csv')

In [4]:
train_transaction = pd.read_csv('/home/other/kaggle/data/train_transaction.csv')
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# for col in train_transaction.columns:
#     if type(train_transaction[col].iloc[0]) == np.float64:
#         train_transaction[col] = train_transaction[col].astype(np.float16)
# train_transaction.to_csv('/kaggle/low_dim/train_transaction.csv')

In [6]:
target = 'isFraud'

In [7]:
train_identity.shape, train_transaction.shape

((144233, 41), (590540, 394))

In [8]:
df = pd.merge(train_identity, train_transaction, on='TransactionID', how='outer')
df.shape

(590540, 434)

In [9]:
# drop columns where more than 50% of NaNs

drop_cols = df.isna().sum()[df.isna().sum() > (df.shape[0] / 2)].index.tolist()
df.drop(drop_cols, axis=1, inplace=True)
df.shape

(590540, 220)

In [10]:
cat_cols_transaction = ['ProductCD',  
                        'addr1', 
                        'addr2',
                        'P_emaildomain',
                        'R_emaildomain']
cat_cols_transaction.extend(['card{}'.format(i) for i in range(1, 7)])
cat_cols_transaction.extend(['M{}'.format(i) for i in range(1, 10)])

cat_cols_identity = ['DeviceType',
                     'DeviceInfo']
cat_cols_identity.extend(['id_{}'.format(i) for i in range(12, 39)])

print('cat_cols_transaction: ', cat_cols_transaction)
print('cat_cols_identity: ', cat_cols_identity)

cat_cols_transaction:  ['ProductCD', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']
cat_cols_identity:  ['DeviceType', 'DeviceInfo', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']


In [11]:
cat_cols_tr = []
for col in cat_cols_transaction:
    if col not in drop_cols:
        cat_cols_tr.append(col)

cat_cols_id = []
for col in cat_cols_identity:
    if col not in drop_cols:
        cat_cols_id.append(col)
        
print('cat_cols_tr: ', cat_cols_tr)
print('cat_cols_id: ', cat_cols_id)

cat_cols_tr:  ['ProductCD', 'addr1', 'addr2', 'P_emaildomain', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'M1', 'M2', 'M3', 'M4', 'M6']
cat_cols_id:  []


In [12]:
# for col in cat_cols_id:
#     if df[col].nunique() < 3:
#         print(col, df[col].unique())

In [13]:
for col in cat_cols_tr:
    if df[col].nunique() < 3:
        print(col, df[col].unique())

M1 [nan 'T' 'F']
M2 [nan 'T' 'F']
M3 [nan 'T' 'F']
M6 [nan 'T' 'F']


In [14]:
# train_identity[cat_cols_id].fillna(-1, inplace=True)
df[cat_cols_tr].fillna(-1, inplace=True)

In [15]:
# not_cat_cols_id

In [16]:
# cat_cols_id

In [17]:
# not_cat_cols_id = [col for col in df.columns if col not in cat_cols_id and col != target]
not_cat_cols_tr = [col for col in df.columns if col not in cat_cols_tr and col != target]

In [18]:
# df[not_cat_cols_id].apply(lambda x: x.fillna(x.values.mean(), inplace=True))
df[not_cat_cols_tr].apply(lambda x: x.fillna(x.values.mean(), inplace=True))

TransactionID     None
TransactionDT     None
TransactionAmt    None
C1                None
C2                None
                  ... 
V317              None
V318              None
V319              None
V320              None
V321              None
Length: 204, dtype: object

In [19]:
df['V317'].unique()

array([   0.        ,  100.        ,   48.63809967, ..., 3816.        ,
        680.5       , 1908.        ])

TransactionDT - timedelta. Suggest that it is the number of seconds since 1st January 2017

In [20]:
df['timestamp'] = pd.to_datetime(df['TransactionDT'], unit='s')
df[['TransactionDT', 'timestamp']]

,TransactionDT,timestamp
0,86506,1970-01-02 00:01:46
1,86535,1970-01-02 00:02:15
2,86549,1970-01-02 00:02:29
3,86555,1970-01-02 00:02:35
4,86620,1970-01-02 00:03:40
...,...,...
590535,15811047,1970-07-02 23:57:27
590536,15811049,1970-07-02 23:57:29
590537,15811079,1970-07-02 23:57:59
590538,15811088,1970-07-02 23:58:08


In [21]:
df['timestamp'].iloc[0].year

1970

In [22]:
for x in df['timestamp']:
    if x.year != 1970:
        print(x)

In [23]:
start_date = df['timestamp'].iloc[0].date()
i = 0 
j = 0
date_month_agg_not_sort = []
while i < len(df):
    #print(df['timestamp'].iloc[i].date(), start_date, df['timestamp'].iloc[i].date() == start_date)
    if df['timestamp'].iloc[i].date() == start_date:
        date_month_agg_not_sort.append(j)
    else:
        j += 1
        start_date = df['timestamp'].iloc[i].date()
        date_month_agg_not_sort.append(j)
    i += 1
len(date_month_agg_not_sort) == len(df)

True

In [24]:
df['date_month_agg_not_sort'] = date_month_agg_not_sort
df[['timestamp', 'date_month_agg_not_sort']]

,timestamp,date_month_agg_not_sort
0,1970-01-02 00:01:46,0
1,1970-01-02 00:02:15,0
2,1970-01-02 00:02:29,0
3,1970-01-02 00:02:35,0
4,1970-01-02 00:03:40,0
...,...,...
590535,1970-07-02 23:57:27,363
590536,1970-07-02 23:57:29,363
590537,1970-07-02 23:57:59,363
590538,1970-07-02 23:58:08,363


In [25]:
df.sort_values(by=['timestamp'], inplace=True)
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V314,V315,V316,V317,V318,V319,V320,V321,timestamp,date_month_agg_not_sort
144233,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,1970-01-02 00:00:00,182
144234,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1970-01-02 00:00:01,182
144235,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1970-01-02 00:01:09,182
144236,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,1970-01-02 00:01:39,182
0,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1970-01-02 00:01:46,0


All rows are belonged to one year

In [26]:
start_date = df['timestamp'].iloc[0].date()
i = 0 
j = 0
date_month_agg = []
while i < len(df):
    #print(df['timestamp'].iloc[i].date(), start_date, df['timestamp'].iloc[i].date() == start_date)
    if df['timestamp'].iloc[i].date() == start_date:
        date_month_agg.append(j)
    else:
        j += 1
        start_date = df['timestamp'].iloc[i].date()
        date_month_agg.append(j)
    i += 1
len(date_month_agg) == len(df)

True

In [27]:
df['date_month_agg'] = date_month_agg
df[['timestamp', 'date_month_agg', 'date_month_agg_not_sort']]

,timestamp,date_month_agg,date_month_agg_not_sort
144233,1970-01-02 00:00:00,0,182
144234,1970-01-02 00:00:01,0,182
144235,1970-01-02 00:01:09,0,182
144236,1970-01-02 00:01:39,0,182
0,1970-01-02 00:01:46,0,0
...,...,...,...
590535,1970-07-02 23:57:27,181,363
590536,1970-07-02 23:57:29,181,363
590537,1970-07-02 23:57:59,181,363
590538,1970-07-02 23:58:08,181,363


In [28]:
# Encoding of categorical features

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[cat_cols_tr] = df[cat_cols_tr].apply(lambda x: le.fit_transform(x.astype(str)))

In [29]:
df[cat_cols_tr].head()

,ProductCD,addr1,addr2,P_emaildomain,card1,card2,card3,card4,card5,card6,M1,M2,M3,M4,M6
144233,4,166,65,32,3417,500,42,1,38,1,1,1,1,2,1
144234,4,173,65,16,7922,303,42,2,2,1,2,2,2,0,1
144235,4,178,65,36,9383,389,42,4,58,2,1,1,1,0,0
144236,4,282,65,54,6991,466,42,2,14,2,2,2,2,0,0
0,1,241,65,16,9262,413,42,2,2,1,2,2,2,3,2


In [30]:
df.drop('timestamp', axis=1, inplace=True)

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.drop(target, axis=1), 
                                                  df[target], 
                                                  test_size=0.33, 
                                                  shuffle=True, 
                                                  stratify=df[target])

In [32]:
X_train.values

array([[3.295252e+06, 7.669619e+06, 1.148000e+01, ..., 0.000000e+00,
        2.690000e+02, 8.700000e+01],
       [3.024389e+06, 9.307360e+05, 4.895000e+01, ..., 0.000000e+00,
        1.910000e+02, 9.000000e+00],
       [3.309572e+06, 8.011039e+06, 4.500000e+01, ..., 3.000000e+01,
        9.100000e+01, 9.100000e+01],
       ...,
       [3.335591e+06, 8.607320e+06, 4.600000e+01, ..., 4.600000e+01,
        2.800000e+02, 9.800000e+01],
       [3.000268e+06, 3.901950e+05, 1.000000e+02, ..., 0.000000e+00,
        3.000000e+00, 3.000000e+00],
       [3.031398e+06, 1.057369e+06, 1.515800e+01, ..., 0.000000e+00,
        1.100000e+01, 1.100000e+01]])

In [33]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import optuna
from sklearn.model_selection import cross_validate, StratifiedKFold

In [34]:
def objective(trial):
    parameters = {
        'n_estimators': trial.suggest_int("n_estimators", 80, 401, step=10),
        'learning_rate': trial.suggest_float("learning_rate", low=0.05, high=0.46, step=0.05),
        'num_leaves': trial.suggest_int("num_leaves", low=2, high=101, step=10),
        'max_depth': trial.suggest_int("max_depth", low=-1, high=41, step=4),  
        'boosting': trial.suggest_categorical("boosting", choices=['gbdt', 'dart']) 
    }
    
    model = LGBMClassifier(**parameters)
    
    
    cv_res_lgbm = cross_validate(model,
                                 X_train, 
                                 y_train, 
                                 scoring=["roc_auc",
                                          "accuracy",
                                          "f1",
                                          "balanced_accuracy",
                                          "precision",
                                          "recall",
                                          "average_precision"],
                                 return_train_score=True,
                                 cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=17))
 
    for score in cv_res_lgbm.keys():
        if score.startswith('train_') or score.startswith('test_'):
            trial.set_user_attr(score, cv_res_lgbm[score])
            trial.set_user_attr('mean_{}'.format(score), cv_res_lgbm[score].mean())

    return cv_res_lgbm['test_roc_auc'].mean()

In [35]:
default_params = {'n_estimators': 100, 
                  'learning_rate': 0.1, 
                  'num_leaves': 31, 
                  'max_depth': -1, 
                  'boosting': 'gbdt'}
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
study.enqueue_trial(default_params)
study.optimize(objective, n_trials=100)
best_trial = study.best_trial
print('Best parameters LightGBM:', best_trial.params)

[I 2021-01-06 20:13:59,314] A new study created in memory with name: no-name-88a5f315-6f78-4e3f-b65e-305bfe82499f
[I 2021-01-06 20:14:56,631] Trial 0 finished with value: 0.9259435947189975 and parameters: {'n_estimators': 100, 'learning_rate': 0.1, 'num_leaves': 31, 'max_depth': -1, 'boosting': 'gbdt'}. Best is trial 0 with value: 0.9259435947189975.
[I 2021-01-06 20:16:17,554] Trial 1 finished with value: 0.9069146745397099 and parameters: {'n_estimators': 130, 'learning_rate': 0.1, 'num_leaves': 32, 'max_depth': 27, 'boosting': 'dart'}. Best is trial 0 with value: 0.9259435947189975.
[I 2021-01-06 20:20:18,821] Trial 2 finished with value: 0.8014000731198727 and parameters: {'n_estimators': 290, 'learning_rate': 0.45, 'num_leaves': 62, 'max_depth': 27, 'boosting': 'dart'}. Best is trial 0 with value: 0.9259435947189975.
[I 2021-01-06 20:21:54,501] Trial 3 finished with value: 0.8971800213301526 and parameters: {'n_estimators': 230, 'learning_rate': 0.25, 'num_leaves': 92, 'max_depth

Best parameters LightGBM: {'n_estimators': 400, 'learning_rate': 0.1, 'num_leaves': 92, 'max_depth': 19, 'boosting': 'gbdt'}


In [36]:
clf = LGBMClassifier(n_estimators=best_trial.params['n_estimators'], 
                     learning_rate=best_trial.params['learning_rate'],
                     num_leaves=best_trial.params['num_leaves'], 
                     max_depth=best_trial.params['max_depth'], 
                     boosting=best_trial.params['boosting'])
clf.fit(X_train.values, y_train.values)
proba = clf.predict_proba(X_val.values)[:, 1]

roc_auc_score(y_val, proba)

0.9647163810440943

# Make a prediction for test dataset

In [37]:
test_identity = pd.read_csv('/home/other/kaggle/data/test_identity.csv', names=train_identity.columns)
test_identity.drop(0, axis=0, inplace=True)
test_identity.shape

(141907, 41)

In [38]:
# for col in test_identity.columns:
#     if type(test_identity[col].iloc[0]) == np.float64:
#         test_identity[col] = test_identity[col].astype(np.float16)
# test_identity.to_csv('/kaggle/low_dim/test_identity.csv')

In [39]:
tr_cols = list(train_transaction.columns)
tr_cols.remove(target)
#tr_cols

In [40]:
test_transaction = pd.read_csv('/home/other/kaggle/data/test_transaction.csv', names=tr_cols)
test_transaction.drop(0, axis=0, inplace=True)
test_transaction.shape

(506691, 393)

In [41]:
# for col in test_transaction.columns:
#     if type(test_transaction[col].iloc[0]) == np.float64:
#         test_transaction[col] = test_transaction[col].astype(np.float16)
# test_transaction.to_csv('/kaggle/low_dim/test_transaction.csv')

In [42]:
test = pd.merge(test_identity, test_transaction, on='TransactionID', how='right')
test.shape

(506691, 433)

In [43]:
test.drop(drop_cols, axis=1, inplace=True)
test.shape

(506691, 219)

In [44]:
test[cat_cols_tr].fillna(-1, inplace=True)
test[not_cat_cols_tr].apply(lambda x: x.fillna(x.values.astype(np.float32).mean(), inplace=True))

test['timestamp'] = pd.to_datetime(test['TransactionDT'], unit='s')
test[['TransactionDT', 'timestamp']]

,TransactionDT,timestamp
0,18403224,1970-08-02 00:00:24
1,18403263,1970-08-02 00:01:03
2,18403310,1970-08-02 00:01:50
3,18403310,1970-08-02 00:01:50
4,18403317,1970-08-02 00:01:57
...,...,...
506686,34214279,1971-01-31 23:57:59
506687,34214287,1971-01-31 23:58:07
506688,34214326,1971-01-31 23:58:46
506689,34214337,1971-01-31 23:58:57


In [45]:
start_date = pd.to_datetime(df['TransactionDT'].iloc[-1], unit='s')
print(start_date)
i = 0 
j = date_month_agg_not_sort[-1] + 1
date_month_agg_not_sort = []
while i < len(test):
    #print(df['timestamp'].iloc[i].date(), start_date, df['timestamp'].iloc[i].date() == start_date)
    if test['timestamp'].iloc[i].date() == start_date:
        date_month_agg_not_sort.append(j)
    else:
        j += 1
        start_date = test['timestamp'].iloc[i].date()
        date_month_agg_not_sort.append(j)
    i += 1
len(date_month_agg_not_sort) == len(test)

1970-07-02 23:58:51


True

In [46]:
test['date_month_agg_not_sort'] = date_month_agg_not_sort
test[['timestamp', 'date_month_agg_not_sort']]

,timestamp,date_month_agg_not_sort
0,1970-08-02 00:00:24,365
1,1970-08-02 00:01:03,365
2,1970-08-02 00:01:50,365
3,1970-08-02 00:01:50,365
4,1970-08-02 00:01:57,365
...,...,...
506686,1971-01-31 23:57:59,547
506687,1971-01-31 23:58:07,547
506688,1971-01-31 23:58:46,547
506689,1971-01-31 23:58:57,547


In [47]:
test.sort_values(by=['timestamp'], inplace=True)
test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V314,V315,V316,V317,V318,V319,V320,V321,timestamp,date_month_agg_not_sort
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1970-08-02 00:00:24,365
1,3663550,18403263,49.0,W,4272,111.0,150.0,visa,226.0,debit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1970-08-02 00:01:03,365
2,3663551,18403310,171.0,W,4476,574.0,150.0,visa,226.0,debit,...,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,1970-08-02 00:01:50,365
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,282.5400085449219,282.5400085449219,0.0,0.0,0.0,0.0,0.0,0.0,1970-08-02 00:01:50,365
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,183.8500061035156,67.94999694824219,0.0,0.0,0.0,0.0,0.0,0.0,1970-08-02 00:01:57,365


In [48]:
date_month_agg[-1]

181

In [49]:
start_date = pd.to_datetime(df['TransactionDT'].iloc[-1], unit='s')
print(start_date)
i = 0 
j = date_month_agg[-1] + 1
date_month_agg = []
while i < len(test):
    #print(df['timestamp'].iloc[i].date(), start_date, df['timestamp'].iloc[i].date() == start_date)
    if test['timestamp'].iloc[i].date() == start_date:
        date_month_agg.append(j)
    else:
        j += 1
        start_date = test['timestamp'].iloc[i].date()
        date_month_agg.append(j)
    i += 1
len(date_month_agg) == len(test)

1970-07-02 23:58:51


True

In [50]:
test['date_month_agg'] = date_month_agg
test[['timestamp', 'date_month_agg_not_sort', 'date_month_agg']]

,timestamp,date_month_agg_not_sort,date_month_agg
0,1970-08-02 00:00:24,365,183
1,1970-08-02 00:01:03,365,183
2,1970-08-02 00:01:50,365,183
3,1970-08-02 00:01:50,365,183
4,1970-08-02 00:01:57,365,183
...,...,...,...
506686,1971-01-31 23:57:59,547,365
506687,1971-01-31 23:58:07,547,365
506688,1971-01-31 23:58:46,547,365
506689,1971-01-31 23:58:57,547,365


In [51]:
test[cat_cols_tr] = test[cat_cols_tr].apply(lambda x: le.fit_transform(x.astype(str)))

In [52]:
test.drop('timestamp', axis=1, inplace=True)

In [53]:
test.shape

(506691, 221)

In [54]:
clf1 = LGBMClassifier(n_estimators=best_trial.params['n_estimators'], 
                      learning_rate=best_trial.params['learning_rate'],
                      num_leaves=best_trial.params['num_leaves'], 
                      max_depth=best_trial.params['max_depth'], 
                      boosting=best_trial.params['boosting'])
clf1.fit(df.drop(target, axis=1), df[target])
proba_test = (clf1.predict_proba(test.values)[:, 1] + clf.predict_proba(test.values)[:, 1]) / 2
ans_df = pd.DataFrame(columns=['TransactionID', 'isFraud'])
ans_df['TransactionID'] = test['TransactionID']
ans_df['isFraud'] = proba_test
ans_df

,TransactionID,isFraud
0,3663549,0.000217
1,3663550,0.000588
2,3663551,0.001151
3,3663552,0.000847
4,3663553,0.000688
...,...,...
506686,4170235,0.014065
506687,4170236,0.003949
506688,4170237,0.003067
506689,4170238,0.008886


In [55]:
# ans_df.drop_duplicates('TransactionID', keep='first', inplace=True)
# ans_df

In [56]:
ans_df.to_csv('/home/other/kaggle/submission_5.csv', index=False)

In [57]:
# !kaggle competitions submit -c ieee-fraud-detection -f submission.csv -m "submission_1"